In [1]:
import numpy as np
import osmnx as ox
import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt

import random
import pickle
import pymde
from scipy.sparse.linalg import eigsh
from sklearn.manifold import MDS 

import mlrfit as mf
import lrrouting as ldr

/Users/parshakova.tanya/opt/anaconda3/envs/routing/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(1001)
random.seed(1001)

#  Matrix definition

In [3]:
n = 1000
dim = 50

rank = dim

X = np.random.randn(n, dim)
# X -= np.ones((n, 1)) @ X.sum(axis=0).reshape(1, -1) / n
# assert np.allclose(X.sum(axis=0), np.zeros(dim))

Dist, Dist_sq = ldr.eulidean_dist_matrix(X)

# Classical MDS

In [4]:
X_cmds, G_cmds = ldr.classical_mds(Dist, n)

assert np.allclose(X_cmds @ X_cmds.T, G_cmds)

/Users/parshakova.tanya/opt/anaconda3/envs/routing/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "


In [5]:
X_cmds, G_cmds = ldr.classical_mds(Dist, rank)
assert X_cmds.shape == (n, rank)

In [6]:
D_cmds, D_cmds_sq = ldr.eulidean_dist_matrix(X_cmds)
mf.rel_diff(D_cmds, Dist)

2.2084469132446001e-16

# MDS

In [7]:
mds = MDS(n_components=rank, metric=True, max_iter=3000, dissimilarity='precomputed') 
X_mds = mds.fit_transform(Dist)


D_mds, D_mds_sq = ldr.eulidean_dist_matrix(X_mds)

In [8]:
mf.rel_diff(D_mds, Dist)

0.0031442658502436552

In [9]:
assert X_mds.shape == (n, rank)

# Fast $\ell_2$ implementation

In [10]:
Z, loss, losses = ldr.fast_cc(rank, pi_rows=np.arange(n), pi_rows_c=np.array([], dtype=np.int64), rDist=Dist, symm=True, n_init=5,
                   max_iter=2000, eps=1e-6, verbose=True, freq=500)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


t=0, loss=3.9011745411508434
t=500, loss=0.055425164041466926
t=1000, loss=0.005717794048081303
t=1500, loss=1.8526871964351338e-05
loss=5.999731247081533e-08, np.diff(np.array(losses)).max()=-6.723844515997825e-10


In [11]:
D_mds, D_mds_sq = ldr.eulidean_dist_matrix(Z)
mf.rel_diff(D_mds, Dist)

1.7171372837270025e-05